In [ ]:
# Extract only GLCM features from brain MRI

import cv2
import os
import sys
import glob
import csv
import imageio
from skimage import color
from skimage.transform import resize
from scipy import ndimage
from scipy.stats import kurtosis, skew
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
from skimage.feature import local_binary_pattern, greycomatrix, greycoprops
from skimage.color import label2rgb
from skimage.filters.rank import entropy
from skimage.morphology import disk
import mahotas as mt
import pywt
import pywt.data
from numpy.testing import assert_array_almost_equal


q = "'"

# Image folder path
path = '...'
path_out = '...'
list_img = os.listdir(path) 
# number_files = number of files in the input folder
number_files = len(list_img)
print(number_files)

i = 0
j = 1
# Input image folder is 'yes'
features = [[0 for x in range(9)] for y in range(number_files+1)]
features[0][0] = 'Image'
features[0][1] = 'Cooccurance Matrix'
features[0][2] = 'Contrast' 
features[0][3] = 'Dissimilarity'
features[0][4] = 'Homogeneity'
features[0][5] = 'ASM'
features[0][6] = 'Energy'
features[0][7] = 'Correlation'
features[0][8] = 'Label'


for file in glob.glob(path + '*.jpg'): 
    i = i+1
    print("File - ", i)
    head, tail = os.path.split(file)
    #print(head)
    #print(tail)
    # f_name has the file name with extension i.e. Y1.jpg, Y2.jpg
    f_name = tail
    features[j][0] = tail
    head2, tail2 = f_name.split(".")
    #print(head2)
    # filename has only the name i.e. Y1, Y2
    filename = head2
    #print(filename)
    
    # Read each image in im
    im = imageio.imread(file) 
    
    # Convert im from 3D to 2D
    image = color.rgb2gray(im)

    # Resize each image into height = 300 and width = 250
    img_resized = resize(image, (256, 256))
    
    #------- Texture Features --------------------------------
    img = np.array(img_resized, dtype=np.uint8)

    # -------------- Cooccurance Matrix --------------------------
    g = greycomatrix(img, [1, 2, 3], [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256, normed=True, symmetric=True)
    
    contrast = greycoprops(g, 'contrast')
    dis = greycoprops(g, 'dissimilarity')
    hom = greycoprops(g, 'homogeneity')
    asm = greycoprops(g, 'ASM')
    ene = greycoprops(g, 'energy')
    cor = greycoprops(g, 'correlation')
    
    features[j][1] = np.mean(g)
    features[j][2] = np.mean(contrast)
    features[j][3] = np.mean(dis)
    features[j][4] = np.mean(hom)
    features[j][5] = np.mean(asm)
    features[j][6] = np.mean(ene)
    features[j][7] = np.mean(cor)  
    
    
    if (filename[0] == 'Y'):
        features[j][8] = 1   
    elif (filename[0] == 'N'):
        features[j][8] = 0
    
    
    j = j + 1


with open(path_out + 'GLCM_Features.csv', 'wt', newline ='') as file:
    writer1 = csv.writer(file, delimiter=',')
    #writer.writerow(i for i in header)
    for x in features:
        writer1.writerow(x)
file.close()